In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install geopandas

In [2]:
!pip install pickle5
import pickle5 as pickle

In [3]:
import pandas as pd
import numpy as np

import seaborn as sns
import warnings
warnings.filterwarnings(action='ignore')
import matplotlib.pyplot as plt
import geopandas as gpd

In [4]:
with open('/content/drive/My Drive/금천구_초등학교881m_학원정보_new.pkl', 'rb') as f:  
  aca = pickle.load(f)

In [5]:
with open('/content/drive/My Drive/금천구_초등학교881m_인구.pkl', 'rb') as f:  
  pop = pickle.load(f)

In [9]:
with open('/content/drive/My Drive/금천구_초등학교881m_돌봄시설정보.pkl', 'rb') as f:  
  dol = pickle.load(f)
 
# /content/drive/My Drive/금천구_초등학교881m_돌봄시설정보.pkl

In [18]:
acacnt = 0
for i in range(len(aca['학원개수'])):
    if aca['학원개수'][i] != 0:
        acacnt += 1
acacnt

194

In [27]:
aca.info()
aca.fillna(0,inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 0 to 193
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   gid     194 non-null    object 
 1   행정동코드   194 non-null    int64  
 2   행정동     194 non-null    object 
 3   위도      194 non-null    float64
 4   경도      194 non-null    float64
 5   학원명     191 non-null    object 
 6   학원개수    191 non-null    float64
dtypes: float64(3), int64(1), object(3)
memory usage: 17.1+ KB


In [33]:
tf_aca = [(float(aca['학원개수'][i] + 1)) for i in range(len(aca))]
idf_aca = (len(aca.gid.unique()) / (acacnt+1))
tfidf_aca = [x*idf_aca for x in tf_aca]
aca['학원tfidf'] = tfidf_aca 

In [34]:
aca.tail(20)

,gid,행정동코드,행정동,위도,경도,학원명,학원개수,학원tfidf
174,다사48ab39ab,1154568000,시흥제2동,37.452126,126.916316,"[평강보습학원, 진학학원, 나우누리학원, 늘찬학원, 씨엠에스벽산캠퍼스학원, 제임스입...",21.0,21.887179
175,다사48ab39ba,1154571000,시흥제5동,37.454379,126.916299,"[평강보습학원, 진학학원, 나우누리학원, 늘찬학원, 씨엠에스벽산캠퍼스학원, 제임스입...",23.0,23.876923
176,다사48ab39bb,1154571000,시흥제5동,37.456633,126.916281,"[평강보습학원, 진학학원, 나우누리학원, 늘찬학원, 씨엠에스벽산캠퍼스학원, 제임스입...",25.0,25.866667
177,다사48ba38aa,1154571000,시흥제5동,37.440873,126.919230,0,0.0,0.994872
178,다사48ba38ab,1154571000,시흥제5동,37.443127,126.919212,"[진학학원, 제임스입시영어학원, 영어능률주니어랩학원, 정현학원, 그룹5영어수학학원,...",6.0,6.964103
179,다사48ba38ba,1154568000,시흥제2동,37.445380,126.919195,"[평강보습학원, 진학학원, 나우누리학원, 늘찬학원, 씨엠에스벽산캠퍼스학원, 제임스입...",16.0,16.912821
180,다사48ba38bb,1154568000,시흥제2동,37.447633,126.919178,"[평강보습학원, 진학학원, 나우누리학원, 늘찬학원, 씨엠에스벽산캠퍼스학원, 제임스입...",18.0,18.902564
181,다사48ba39aa,1154568000,시흥제2동,37.449887,126.919160,"[평강보습학원, 진학학원, 나우누리학원, 늘찬학원, 씨엠에스벽산캠퍼스학원, 제임스입...",20.0,20.892308
182,다사48ba39ab,1154568000,시흥제2동,37.452140,126.919143,"[평강보습학원, 진학학원, 나우누리학원, 늘찬학원, 씨엠에스벽산캠퍼스학원, 제임스입...",20.0,20.892308
183,다사48ba39ba,1154568000,시흥제2동,37.454393,126.919125,"[평강보습학원, 진학학원, 나우누리학원, 늘찬학원, 씨엠에스벽산캠퍼스학원, 제임스입...",21.0,21.887179


In [11]:
dol.columns

Index(['gid', '행정동코드', '행정동', '위도', '경도', '키움센터', '키움센터_개수', '키움센터_상시돌봄정원',
       '키움센터_일시돌봄정원', '초등돌봄교실', '초등돌봄교실_개수', '초등돌봄교실_교실개수', '초등돌봄교실_이용인원',
       '지역아동센터', '지역아동센터_개수', '지역아동센터_정원', '방과후아카데미_개수', '방과후아카데미_정원'],
      dtype='object')

In [15]:
pop

,gid,행정동코드,시군구명,행정동,학교명,학교개수,초등학생_인구,30대_인구,40대_인구,전체_인구,경도,위도,geometry,초등생인구비율
0,다사44bb42ba,1154551000,금천구,가산동,[서울영일초등학교],1,0.0,346.0,95.0,684.0,126.876503,37.481217,POINT (126.87650 37.48122),0.000125
1,다사44bb42bb,1154551000,금천구,가산동,[서울영일초등학교],1,0.0,169.0,32.0,415.0,126.876485,37.483470,POINT (126.87648 37.48347),0.000125
2,다사45aa42ab,1154551000,금천구,가산동,[서울영일초등학교],1,0.0,0.0,0.0,0.0,126.879350,37.478979,POINT (126.87935 37.47898),0.000125
3,다사45aa42ba,1154551000,금천구,가산동,[서울영일초등학교],1,0.0,9.0,7.0,74.0,126.879331,37.481232,POINT (126.87933 37.48123),0.000125
4,다사45aa42bb,1154551000,금천구,가산동,[서울영일초등학교],1,0.0,0.0,0.0,0.0,126.879312,37.483485,POINT (126.87931 37.48349),0.000125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,다사48bb39ba,1154568000,금천구,시흥제2동,"[서울금천초등학교, 서울탑동초등학교, 동광초등학교]",3,0.0,0.0,0.0,0.0,126.921952,37.454407,POINT (126.92195 37.45441),0.000125
190,다사48bb39bb,1154568000,금천구,시흥제2동,"[서울금천초등학교, 서울탑동초등학교, 동광초등학교, 서울난향초등학교]",4,0.0,0.0,0.0,0.0,126.921934,37.456661,POINT (126.92193 37.45666),0.000125
191,다사49aa38bb,1154568000,금천구,시흥제2동,"[서울금동초등학교, 동광초등학교]",2,0.0,0.0,0.0,0.0,126.924830,37.447661,POINT (126.92483 37.44766),0.000125
192,다사49aa39aa,1154568000,금천구,시흥제2동,"[서울금동초등학교, 동광초등학교]",2,0.0,0.0,0.0,0.0,126.924813,37.449914,POINT (126.92481 37.44991),0.000125


In [13]:
popcnt = 0
for i in range(len(pop['초등학생_인구'])):
    if pop['초등학생_인구'][i] != 0:
        popcnt += pop['초등학생_인구'][i]
popcnt

8017.0

In [49]:
dol.fillna(0,inplace=True)
dol['총시설수'] = dol['키움센터_개수']+dol['초등돌봄교실_개수']+dol['지역아동센터_개수']+dol['방과후아카데미_개수']
dol['총정원'] = dol['키움센터_상시돌봄정원']+dol['키움센터_일시돌봄정원']+dol['초등돌봄교실_이용인원']+dol['지역아동센터_정원']+dol['방과후아카데미_정원']
dol['돌봄교실이용비율'] = [x/sum(dol['총정원']) for x in dol['총정원']]

In [58]:
# for x in pop['초등학생_인구']:
#   lst = []
#   lst.append(x+1/popcnt)
# lst
pop['초등생인구비율'] = (pop['초등학생_인구']+1) / popcnt
# [(x+1/popcnt) for x in pop['초등학생_인구']]

In [36]:
cnt = 0
for i in range(len(dol['총시설수'])):
    if dol['총시설수'][i] != 0:
        cnt += 1
cnt

194

In [66]:
#  tf_center = [float(all['총학원수'][i]+1) / float(all['총학원수'].sum()) for i in range(len(all))]
#  idf = float(len(all.행정동코드.unique()) / (423))
#  tfidf = [x*idf for x in tf]

In [38]:
tf_center = [(float(dol['총시설수'][i] + 1)) for i in range(len(dol))]
idf_center = (len(dol.gid.unique()) / (cnt+1))
tfidf_center = [x*idf_center for x in tf_center]
dol['센터tfidf']= tfidf_center

# tf_aca = [(float(aca['학원개수'][i] + 1)) for i in range(len(aca))]
# idf_aca = (len(aca.gid.unique()) / (acacnt+1))
# tfidf_aca = [x*idf_aca for x in tf_aca]
# tfidf_aca

In [39]:
dol

,gid,행정동코드,행정동,위도,경도,키움센터,키움센터_개수,키움센터_상시돌봄정원,키움센터_일시돌봄정원,초등돌봄교실,초등돌봄교실_개수,초등돌봄교실_교실개수,초등돌봄교실_이용인원,지역아동센터,지역아동센터_개수,지역아동센터_정원,방과후아카데미_개수,방과후아카데미_정원,총시설수,총정원,돌봄교실이용비율,센터tfidf
0,다사44bb42ba,1154551000,가산동,37.481217,126.876503,0,0.0,0.0,0.0,[서울영일초등학교],1,4.0,77.0,0,0.0,0.0,0,0,1.0,77.0,0.000882,1.989744
1,다사44bb42bb,1154551000,가산동,37.483470,126.876485,0,0.0,0.0,0.0,[서울영일초등학교],1,4.0,77.0,0,0.0,0.0,0,0,1.0,77.0,0.000882,1.989744
2,다사45aa42ab,1154551000,가산동,37.478979,126.879350,0,0.0,0.0,0.0,[서울영일초등학교],1,4.0,77.0,"[누리지역아동센터, 아름다운구로청소년지역아동센터]",2.0,50.0,0,0,3.0,127.0,0.001455,3.979487
3,다사45aa42ba,1154551000,가산동,37.481232,126.879331,0,0.0,0.0,0.0,[서울영일초등학교],1,4.0,77.0,"[누리지역아동센터, 아름다운구로청소년지역아동센터, 구로지역아동센터]",3.0,69.0,0,0,4.0,146.0,0.001672,4.974359
4,다사45aa42bb,1154551000,가산동,37.483485,126.879312,0,0.0,0.0,0.0,[서울영일초등학교],1,4.0,77.0,"[누리지역아동센터, 아름다운구로청소년지역아동센터, 구로지역아동센터, 한사랑지역아동센...",6.0,161.0,0,0,7.0,238.0,0.002726,7.958974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,다사48bb39ba,1154568000,시흥제2동,37.454407,126.921952,0,0.0,0.0,0.0,"[서울금천초등학교, 서울탑동초등학교, 동광초등학교]",3,6.0,98.0,[혜명지역아동복지센터],1.0,40.0,0,0,4.0,138.0,0.001581,4.974359
190,다사48bb39bb,1154568000,시흥제2동,37.456661,126.921934,0,0.0,0.0,0.0,"[서울금천초등학교, 서울탑동초등학교, 동광초등학교, 서울난향초등학교]",4,10.0,200.0,[난향지역아동센터],1.0,23.0,0,0,5.0,223.0,0.002554,5.969231
191,다사49aa38bb,1154568000,시흥제2동,37.447661,126.924830,0,0.0,0.0,0.0,"[서울금동초등학교, 동광초등학교]",2,3.0,55.0,0,0.0,0.0,0,0,2.0,55.0,0.000630,2.984615
192,다사49aa39aa,1154568000,시흥제2동,37.449914,126.924813,0,0.0,0.0,0.0,"[서울금동초등학교, 동광초등학교]",2,3.0,55.0,0,0.0,0.0,0,0,2.0,55.0,0.000630,2.984615


In [59]:
pop.tail(20)

,gid,행정동코드,시군구명,행정동,학교명,학교개수,초등학생_인구,30대_인구,40대_인구,전체_인구,경도,위도,geometry,초등생인구비율,초등생인구비율_
174,다사48ab39ab,1154568000,금천구,시흥제2동,"[서울금동초등학교, 서울금천초등학교, 서울탑동초등학교, 동광초등학교]",4,29.0,76.0,85.0,655.0,126.916316,37.452126,POINT (126.91632 37.45213),0.003742,29.000125
175,다사48ab39ba,1154571000,금천구,시흥제5동,"[서울금동초등학교, 서울신흥초등학교, 서울금천초등학교, 서울탑동초등학교, 동광초등학교]",5,0.0,0.0,0.0,32.0,126.916299,37.454379,POINT (126.91630 37.45438),0.000125,0.000125
176,다사48ab39bb,1154571000,금천구,시흥제5동,"[서울신흥초등학교, 서울금천초등학교, 서울탑동초등학교, 동광초등학교, 서울난향초등학교]",5,0.0,0.0,0.0,0.0,126.916281,37.456633,POINT (126.91628 37.45663),0.000125,0.000125
177,다사48ba38aa,1154571000,금천구,시흥제5동,[서울금동초등학교],1,0.0,0.0,0.0,0.0,126.919230,37.440873,POINT (126.91923 37.44087),0.000125,0.000125
178,다사48ba38ab,1154571000,금천구,시흥제5동,[서울금동초등학교],1,0.0,0.0,0.0,0.0,126.919212,37.443127,POINT (126.91921 37.44313),0.000125,0.000125
179,다사48ba38ba,1154568000,금천구,시흥제2동,"[서울금동초등학교, 서울탑동초등학교, 동광초등학교]",3,0.0,0.0,0.0,0.0,126.919195,37.445380,POINT (126.91919 37.44538),0.000125,0.000125
180,다사48ba38bb,1154568000,금천구,시흥제2동,"[서울금동초등학교, 서울탑동초등학교, 동광초등학교]",3,349.0,319.0,750.0,3497.0,126.919178,37.447633,POINT (126.91918 37.44763),0.043657,349.000125
181,다사48ba39aa,1154568000,금천구,시흥제2동,"[서울금동초등학교, 서울금천초등학교, 서울탑동초등학교, 동광초등학교]",4,263.0,545.0,775.0,4582.0,126.919160,37.449887,POINT (126.91916 37.44989),0.032930,263.000125
182,다사48ba39ab,1154568000,금천구,시흥제2동,"[서울금동초등학교, 서울금천초등학교, 서울탑동초등학교, 동광초등학교]",4,104.0,404.0,384.0,2862.0,126.919143,37.452140,POINT (126.91914 37.45214),0.013097,104.000125
183,다사48ba39ba,1154568000,금천구,시흥제2동,"[서울금동초등학교, 서울금천초등학교, 서울탑동초등학교, 동광초등학교]",4,28.0,146.0,123.0,1329.0,126.919125,37.454393,POINT (126.91913 37.45439),0.003617,28.000125


In [62]:
df1 = pd.merge(aca,pop,on=['gid'])
df2 = pd.merge(df1,dol,on=['gid'])
df2.columns

Index(['gid', '행정동코드_x', '행정동_x', '위도_x', '경도_x', '학원명', '학원개수', '학원tfidf',
       '행정동코드_y', '시군구명', '행정동_y', '학교명', '학교개수', '초등학생_인구', '30대_인구',
       '40대_인구', '전체_인구', '경도_y', '위도_y', 'geometry', '초등생인구비율', '초등생인구비율_',
       '행정동코드', '행정동', '위도', '경도', '키움센터', '키움센터_개수', '키움센터_상시돌봄정원',
       '키움센터_일시돌봄정원', '초등돌봄교실', '초등돌봄교실_개수', '초등돌봄교실_교실개수', '초등돌봄교실_이용인원',
       '지역아동센터', '지역아동센터_개수', '지역아동센터_정원', '방과후아카데미_개수', '방과후아카데미_정원', '총시설수',
       '총정원', '돌봄교실이용비율', '센터tfidf'],
      dtype='object')

In [65]:
df2 = df2[['gid', '행정동코드_x', '행정동_x', '위도_x', '경도_x', '학원명', '학원개수', '학원tfidf',
      '학교명', '학교개수', '초등학생_인구', '30대_인구',
       '40대_인구', '전체_인구', '초등생인구비율','키움센터', '키움센터_개수', '키움센터_상시돌봄정원',
       '키움센터_일시돌봄정원', '초등돌봄교실', '초등돌봄교실_개수', '초등돌봄교실_교실개수', '초등돌봄교실_이용인원',
       '지역아동센터', '지역아동센터_개수', '지역아동센터_정원', '방과후아카데미_개수', '방과후아카데미_정원', '총시설수',
       '총정원', '돌봄교실이용비율', '센터tfidf']]

In [66]:
df2.columns = ['gid', '행정동코드', '행정동', '위도', '경도', '학원명', '학원개수', '학원tfidf',
      '학교명', '학교개수', '초등학생_인구', '30대_인구',
       '40대_인구', '전체_인구', '초등생인구비율','키움센터', '키움센터_개수', '키움센터_상시돌봄정원',
       '키움센터_일시돌봄정원', '초등돌봄교실', '초등돌봄교실_개수', '초등돌봄교실_교실개수', '초등돌봄교실_이용인원',
       '지역아동센터', '지역아동센터_개수', '지역아동센터_정원', '방과후아카데미_개수', '방과후아카데미_정원', '총시설수',
       '총정원', '돌봄교실이용비율', '센터tfidf']

In [69]:
df2['수요지수'] = df2['센터tfidf'] * df2['초등생인구비율']* df2['돌봄교실이용비율'] / df2['학원tfidf']
df2

,gid,행정동코드,행정동,위도,경도,학원명,학원개수,학원tfidf,학교명,학교개수,초등학생_인구,30대_인구,40대_인구,전체_인구,초등생인구비율,키움센터,키움센터_개수,키움센터_상시돌봄정원,키움센터_일시돌봄정원,초등돌봄교실,초등돌봄교실_개수,초등돌봄교실_교실개수,초등돌봄교실_이용인원,지역아동센터,지역아동센터_개수,지역아동센터_정원,방과후아카데미_개수,방과후아카데미_정원,총시설수,총정원,돌봄교실이용비율,센터tfidf,수요지수
0,다사44bb42ba,1154551000,가산동,37.481217,126.876503,"[천재교과서밀크티초등원격학원, 메가수학화상교육원격학원]",2.0,2.984615,[서울영일초등학교],1,0.0,346.0,95.0,684.0,0.000125,0,0.0,0.0,0.0,[서울영일초등학교],1,4.0,77.0,0,0.0,0.0,0,0,1.0,77.0,0.000882,1.989744,7.334380e-08
1,다사44bb42bb,1154551000,가산동,37.483470,126.876485,"[천재교과서밀크티초등원격학원, 외대영어전문학원, 이아이이(EiE)뉴스터디구로구일학원]",3.0,3.979487,[서울영일초등학교],1,0.0,169.0,32.0,415.0,0.000125,0,0.0,0.0,0.0,[서울영일초등학교],1,4.0,77.0,0,0.0,0.0,0,0,1.0,77.0,0.000882,1.989744,5.500785e-08
2,다사45aa42ab,1154551000,가산동,37.478979,126.879350,"[천재교과서밀크티초등원격학원, 메가수학화상교육원격학원]",2.0,2.984615,[서울영일초등학교],1,0.0,0.0,0.0,0.0,0.000125,0,0.0,0.0,0.0,[서울영일초등학교],1,4.0,77.0,"[누리지역아동센터, 아름다운구로청소년지역아동센터]",2.0,50.0,0,0,3.0,127.0,0.001455,3.979487,2.419393e-07
3,다사45aa42ba,1154551000,가산동,37.481232,126.879331,"[천재교과서밀크티초등원격학원, 메가수학화상교육원격학원]",2.0,2.984615,[서울영일초등학교],1,0.0,9.0,7.0,74.0,0.000125,0,0.0,0.0,0.0,[서울영일초등학교],1,4.0,77.0,"[누리지역아동센터, 아름다운구로청소년지역아동센터, 구로지역아동센터]",3.0,69.0,0,0,4.0,146.0,0.001672,4.974359,3.476686e-07
4,다사45aa42bb,1154551000,가산동,37.483485,126.879312,"[신대원학원, 뉴탑클래스학원, 천재교과서밀크티초등원격학원, 외대영어전문학원, 메가수...",6.0,6.964103,[서울영일초등학교],1,0.0,0.0,0.0,0.0,0.000125,0,0.0,0.0,0.0,[서울영일초등학교],1,4.0,77.0,"[누리지역아동센터, 아름다운구로청소년지역아동센터, 구로지역아동센터, 한사랑지역아동센...",6.0,161.0,0,0,7.0,238.0,0.002726,7.958974,3.886269e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,다사48bb39ba,1154568000,시흥제2동,37.454407,126.921952,"[평강보습학원, 진학학원, 나우누리학원, 제임스입시영어학원, 영어능률주니어랩학원, ...",14.0,14.923077,"[서울금천초등학교, 서울탑동초등학교, 동광초등학교]",3,0.0,0.0,0.0,0.0,0.000125,0,0.0,0.0,0.0,"[서울금천초등학교, 서울탑동초등학교, 동광초등학교]",3,6.0,98.0,[혜명지역아동복지센터],1.0,40.0,0,0,4.0,138.0,0.001581,4.974359,6.572366e-08
190,다사48bb39bb,1154568000,시흥제2동,37.456661,126.921934,"[영어능률주니어랩학원, 그룹5영어수학학원, 해법영어셀파수학학원, 명문보습학원, 아프...",9.0,9.948718,"[서울금천초등학교, 서울탑동초등학교, 동광초등학교, 서울난향초등학교]",4,0.0,0.0,0.0,0.0,0.000125,0,0.0,0.0,0.0,"[서울금천초등학교, 서울탑동초등학교, 동광초등학교, 서울난향초등학교]",4,10.0,200.0,[난향지역아동센터],1.0,23.0,0,0,5.0,223.0,0.002554,5.969231,1.911701e-07
191,다사49aa38bb,1154568000,시흥제2동,37.447661,126.924830,"[진학학원, 영어능률주니어랩학원, 그룹5영어수학학원, 해법영어셀파수학학원]",4.0,4.974359,"[서울금동초등학교, 동광초등학교]",2,0.0,0.0,0.0,0.0,0.000125,0,0.0,0.0,0.0,"[서울금동초등학교, 동광초등학교]",2,3.0,55.0,0,0.0,0.0,0,0,2.0,55.0,0.000630,2.984615,4.714958e-08
192,다사49aa39aa,1154568000,시흥제2동,37.449914,126.924813,"[진학학원, 영어능률주니어랩학원, 눈높이러닝센터시흥벽산학원, 그룹5영어수학학원, 해...",5.0,5.969231,"[서울금동초등학교, 동광초등학교]",2,0.0,0.0,0.0,0.0,0.000125,0,0.0,0.0,0.0,"[서울금동초등학교, 동광초등학교]",2,3.0,55.0,0,0.0,0.0,0,0,2.0,55.0,0.000630,2.984615,3.929132e-08


In [70]:
pt = np.percentile(df2[['수요지수']],[i*10 for i in range(1,10)]).tolist()
pt

[1.7645154721239348e-07,
 3.639090602031518e-07,
 5.639008574818843e-07,
 3.9859019848870805e-06,
 1.0654327493873163e-05,
 1.9461527948226937e-05,
 2.6277927530729688e-05,
 3.419814668787121e-05,
 5.109246305577995e-05]

In [71]:
df2['percentile'] = 0
df2.loc[(df2['수요지수'] <= pt[0]),'percentile'] = '1'
df2.loc[(df2['수요지수'] > pt[0]) & (df2['수요지수'] <= pt[1]),'percentile'] = '2'
df2.loc[(df2['수요지수'] > pt[1]) & (df2['수요지수'] <= pt[2]),'percentile'] = '3'
df2.loc[(df2['수요지수'] > pt[2]) & (df2['수요지수'] <= pt[3]),'percentile'] = '4'
df2.loc[(df2['수요지수'] > pt[3]) & (df2['수요지수'] <= pt[4]),'percentile'] = '5'
df2.loc[(df2['수요지수'] > pt[4]) & (df2['수요지수'] <= pt[5]),'percentile'] = '6'
df2.loc[(df2['수요지수'] > pt[5]) & (df2['수요지수'] <= pt[6]),'percentile'] = '7'
df2.loc[(df2['수요지수'] > pt[6]) & (df2['수요지수'] <= pt[7]),'percentile'] = '8'
df2.loc[(df2['수요지수'] > pt[7]) & (df2['수요지수'] <= pt[8]),'percentile'] = '9'
df2.loc[(df2['수요지수'] > pt[8]),'percentile'] = '10'

In [73]:
df2.to_pickle('금천구_수요지수.pkl')